# Codice per la manipolazione di dati estratti dalle API v2 di Twitter

## Funzionalità

##### Estrazione di risposte a tweet
##### Estrazione di interazione utente-utente con salvataggio di timestamp e testo del tweet
##### Estrazione di tweet a partire da nome utente con l'uso di Twint (https://github.com/twintproject/twint)
##### Creazione di csv con i dati elaborati

In [1]:
# read data
import numpy as np
import ijson
import pandas as pd
from datetime import datetime
import time

import glob
import json
import csv
from NormalizeTweets import normalizeTweet

# generazione embedding
import gensim
import gensim.downloader as api

In [ ]:
# path dei dati
singleline_json = "C:\\Users\\federico\\Documents\\MEGA\\UNI\\2-ML\\Progetto\\ijson\\singleline.json"
table_json = "C:\\Users\\federico\\Documents\\MEGA\\UNI\\2-ML\\Progetto\\results.json"
table_json_nb = "D:\\Progetto\\qatar150k2411.json"
table_json_results = "results.json"
table_json_nb_70k = "D:\\Progetto\\qatar70k1911.json"
table_json_theirs = "D:\\row_datsets\\theirs0401-1001.json"

In [2]:
# conversione data in stringa -> timestamp
def dateToTimestampFromTwitter(date):
    dt = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ')
    return time.mktime(dt.timetuple())


def dateToTimestamp(date):
    dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return time.mktime(dt.timetuple())

In [13]:
# modello doc2vec per la generazione degli embedding
dataset = api.load("text8")
data = [d for d in dataset]

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_training = list(tagged_document(data))

model = gensim.models.doc2vec.Doc2Vec(vector_size=768, min_count=2, epochs=30)

model.build_vocab(data_training)

# training del modello
model.train(data_training, total_examples=model.corpus_count, epochs=model.epochs)

In [14]:
# funzione per la generazione degli embedding a partire dal testo (array di parole)
def getDoc2VecEmbeddingGensim(text):
    return model.infer_vector(text)

In [ ]:
# alternativa al doc2vec: glove-twitter-200, glove-twitter-100, ... ma produce embedding di dimensione fissa

#modelGloveTwitter = api.load("glove-twitter-25")

#def getEmbeddingGloveTwitterGensim(text):
#    return modelGloveTwitter.get_mean_vector(text)

In [ ]:
# creazione del dataframe contenente le informazioni necessarie per l'elaborazione a partire dai dati estratti da twitter

tweetCount = 0
df = pd.DataFrame(columns=["author_id", "tweet_id", "conversation_id", "timestamp", "text"])

with open(table_json_theirs, 'rb') as input_file:
    parser = ijson.parse(input_file, multiple_values=True)
    
    curr_author = 0
    curr_tweet = 0
    curr_text = ''
    curr_date = ''
    curr_conversation = 0
    
    curr_user_id = 0   
    curr_username = ''

    conversation2tweets = {}
    tweet2author = {}
    tweet2text = {}
    tweet2date = {}
    tweetIds = set()
    authors = set()
       
    for prefix, event, value in parser:
        if prefix=="includes.tweets.item.text":# or prefix=="data.item.text":
            if curr_text != '' and curr_tweet!=0:
                df.loc[len(df.index)] = [curr_author, curr_tweet, curr_conversation, curr_date, curr_text]
            curr_text = value
        if prefix=="includes.tweets.item.author_id":# or prefix=="data.item.author_id":
            curr_author = value
            authors.add(value)
        if prefix=="includes.tweets.item.id":# or prefix=="data.item.id":
            #tweetIds.add(value)
            tweetCount += 1
            curr_tweet = value
            tweet2author[value] = curr_author
            tweet2text[value] = curr_text
        if prefix=="includes.tweets.item.conversation_id":# or prefix=="data.item.conversation_id":
            #tweetIds.add(value)
            curr_conversation = value
            """if value != curr_tweet:
                if conversation2tweets.get(value) == None:
                    values = []
                    values.append(curr_tweet)
                    conversation2tweets[value] = values
                else:
                    values = conversation2tweets.get(value)
                    values.append(curr_tweet)"""
        if prefix=="includes.tweets.item.created_at":# or prefix=="data.item.created_at":
            curr_date = dateToTimestampFromTwitter(value)
            tweet2date[curr_tweet] = curr_date

df.to_json('df_withauthorid.json')

In [ ]:
# salvataggio delle corrispondenze user_id <-> username per non effettuare altre chiamate per recuperare questa informazione

df_users = pd.DataFrame(columns=["user_id", "username"])
with open(table_json_nb, 'rb') as input_file:
    parser = ijson.parse(input_file, multiple_values=True)

    curr_id = 0   
    curr_username = ''
    
    for prefix, event, value in parser:
        if prefix=="data.item.entities.mentions.item.username":# or prefix=="data.item.text":
            if curr_id != 0 and curr_username !='':
                df_users.loc[len(df.index)] = [curr_id, curr_username]
            curr_username = value
        if prefix=="data.item.entities.mentions.item.id":# or prefix=="data.item.text":
            curr_id = value
        

df_users.to_json('df_authors_username.json')

In [ ]:
# generazione degli indici da salvare nel dataset finale e salvataggio corrispondenza user_index <-> user_id
count = 0
user_idxs = {}
for user_id in df['author_id']:
    user_idxs[user_id] = count
    count+=1

In [ ]:
# generazione degli indici per i tweet per ricavare l'autore nel dataset finale e salvataggio corrispondenza tweet_index <-> tweet_id
cnt = 0
tweets_ids = {}
for tweet_id in df['tweet_id']:
    tweets_ids[tweet_id] = cnt
    cnt+=1

In [ ]:
# generazione lista dei tweet ricavati per filtrare quelli di cui non si hanno abbastanza informazioni
tweets_ids_list = []
for tweet_id in df['tweet_id']:
    tweets_ids_list.append(tweet_id)

In [ ]:
# filtraggio
df_filtered = df[df['conversation_id'].isin(tweets_ids_list)]

In [ ]:
# corrispondenza tweet <-> autore
tweet2author = {}
c = 0
for user_id, tweet_id in zip(df_filtered['user_id'], df_filtered['tweet_id']):
    tweet2author[tweet_id] = user_id

In [ ]:
# salvataggio del dataset finale contenente id_autore_tweet_risposta, id_autore_tweet_iniziale, timestamp_tweet, testo_tweet
df_final = pd.DataFrame(columns=["user_id", "item_id", "timestamp", "text"])
errors = 0
for index, row in df_filtered.iterrows():
    try:
        df_final.loc[len(df.index)] = [row['user_id'], tweet2author[row['conversation_id']], time.mktime(row['timestamp'].timetuple()), row['text']]
    except:
        errors+=1
print(str(errors))

In [ ]:
# salvataggio df per estrazione statistiche
df_final.to_json('df_final.json')

In [ ]:
# ordinamento tweet per timestamp, normalizzazione valori timestamp e salvataggio dataset in csv (con normalizzazione del testo)
df_final.sort_values(by='timestamp')
df_final['timestamp'] = df_final['timestamp'] - df_final['timestamp'].min()

start = time.time()
with open('D:\dataset_input.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id','item_id','timestamp', 'state_label','comma_separated_list_of_features'])
    for index, row in df_final.iterrows():
        try:
            filewriter.writerow([row.user_id, row.item_id, row.timestamp, 0, *getDoc2VecEmbeddingGensim(normalizeTweet(row.text).split(" "))])
        except Exception as e:
            print(e)
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

In [ ]:
# rimozione righe vuote
# rimozione righe vuote
with open('D:\dataset_input.csv') as input, open('tgn/data/dataset_input.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    for row in csv.reader(input):
        if any(field.strip() for field in row):
            writer.writerow(row)

## SCRAPING PER UTENTE
### Per questa fase è utile memorizzare la corrispondenza indice utente <-> username
###### Con le funzioni usate in questo notebook, è possibile ricavare lo username scorrendo la lista degli user id nel dataframe e controllando la corrispondenza nel dataframe authors_username. è poi possibile generare di nuovo gli indici e ricavare il corrispondente indice (questo è un WORKAROUND che si può evitare salvando direttamente le corrispondenze indice<->id<->username)

In [ ]:

import twint

#read df with author_ids
df_raw = pd.read_json('df_withauthorid.json')

#df with author_id to username correspondance
df_users = pd.read_json("df_authors_username.json")
#delete duplicates
df_users = df_users.drop_duplicates()

#get users' indexes used in the final dataframe
count = 0
user_idxs = {}
for user_id in df_raw['author_id']:
    user_idxs[user_id] = count
    count+=1

errors = 0
counter = 0
start = time.time()

for author_id in df_raw['author_id'].drop_duplicates():
    try:
        # estrazione username
        username = df_users[df_users['user_id']==author_id].drop_duplicates()['username'].values[0]
        # creazione oggetto twint per lo scraping
        c = twint.Config()
        c.Username = username
        c.Limit = 100 # estrazione di 100 tweet per utente
        c.Output = "tweets/tweets-"+str(user_idxs[author_id])+".json" # salvataggio dei tweet in un file json con nome contenente l'indice dell'utente
        c.Store_json = True
        twint.run.Search(c)
        counter+=1
        print(username + "saved") # log degli utenti salvati
    except Exception as e:
        print(e)
        errors +=1

print("Time elapsed: "+ str(time.time() - start))
print("Saved:" + str(counter))
print("Errors:" + str(errors))

In [15]:
count=0
df_user2text = pd.DataFrame(columns=('user_idx', 'text'))
files = glob.glob('tweets/*', recursive=True)
# per ogni utente, i tweet vengono concatenati e salvati in un dataframe
for single_file in files:
    with open(single_file, 'r') as f:
        user_idx = str(single_file).split('-')[1].split('.')[0]
        parser = ijson.parse(single_file, multiple_values=True, )      
        tweet = ''
        for line in f:
            data = json.loads(line)
            tweet = tweet + ' ' + data[u'tweet']
        df_user2text.loc[len(df_user2text)] = [user_idx, tweet]
        
# per ogni utente si genera l'embedding del testo complessivo e si salva nel dataset, assieme all'indice dell'utente
with open('D:\dataset_utenti.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id', 'comma_separated_list_of_features'])
    for index, row in df_user2text.iterrows():
        try:
            # è necessario che ogni vettore di embedding sia composto da 172 valori (è necessario allenare un nuovo modello doc2vec: model = gensim.models.doc2vec.Doc2Vec(vector_size=172, min_count=2, epochs=30) )
            filewriter.writerow([row.user_idx, *getDoc2VecEmbeddingGensim(normalizeTweet(row.text).split(" "))])
        except Exception as e:
            print(e)
end = time.time()

Reading completed


,user_idx,text
0,100092,🇪🇨❌️⚽️ ¡ECUADOR NO CONOCE LA VICTORIA EN EL H...
1,100419,@VoltInuOfficial $VOLT 🥇🌍⚡⚡⚡⚡⚡⚡⚡⚡ https://t....
2,100511,@aghachurchill @unrulified سلام خیلی خوب کار ...
3,10065,Eat it… Feel it… Love it… Celebrate everyday...
4,100709,¡Lindo inicio de semana para todos! #FelizL...
...,...,...
3102,99833,U.S. charges neo-Nazi leader over plot to att...
3103,99834,#LunesDeGanarSeguidores #mexicana #sexy http...
3104,9991,Your gauge determines your week! Choose wise...
3105,99938,30 anggota @InterClubIndo merayakan kemenanga...


In [16]:
# rimozione righe vuote e salvataggio dataset nel path data (il nome deve essere '{dataName}_nodes.csv')
with open('D:\dataset_utenti.csv') as input, open('tgn/data/dataset_input_nodes.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    for row in csv.reader(input):
        if any(field.strip() for field in row):
            writer.writerow(row)

# Statistiche sui dataset

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_json('df_final.json')

In [ ]:
df_stats = pd.DataFrame(columns=["user_id", "interactions"])
df_stats_asCreator = pd.DataFrame(columns=["user_id", "interactions"])
df_stats_asCreator['user_id'] = df[df['user_id']!=df['item_id']]['user_id'].value_counts().index
df_stats_asCreator['interactions'] = df[df['user_id']!=df['item_id']]['user_id'].value_counts().values
df_stats_asReplyer = pd.DataFrame(columns=["user_id", "interactions"])
df_stats_asReplyer['user_id'] = df[df['user_id']!=df['item_id']]['item_id'].value_counts().index
df_stats_asReplyer['interactions'] = df[df['user_id']!=df['item_id']]['item_id'].value_counts().values

df_stats = pd.concat([df_stats_asCreator, df_stats_asReplyer])\
       .groupby('user_id')['interactions']\
       .sum().reset_index()

In [ ]:
df_stats['interactions'].plot(kind='kde', color="darkblue", xlabel="#interactions",figsize=(20,5))
plt.grid(color='blue', linestyle='-', linewidth=0.5, axis="both")

In [ ]:
print('Statistiche numero di interazioni')
print('Minimo: ' + str(df_stats['interactions'].min()))
print('Moda: ' + str(df_stats['interactions'].mode()))
print('Mediana: ' + str(df_stats['interactions'].median()))
print('Media: ' + str(df_stats['interactions'].mean()))
print('Massimo: ' + str(df_stats['interactions'].max()))
print('Totale: ' + str(df_stats['interactions'].sum()))